![intro_banner](./Images/22-4-2.5-Banner.png)

---

The aim of this notebook is to to describe ways to download products on the Jupyter Notebook environment, in order to use them afterwards using Python.

# Download a product directly from the provider

A product is directly accessible on the provider's website. Let's focus on a specific Sentinel 2 product that will be our example for the whole notebook.

For an optical image, the contrast between water and sand is intersting.We choose a product from the *Siwa Oasis* region in Egypt, which offers a strong contrast.  Since 2014, The ESA Sentinel-2 satellite offers optical imagery products that would fit our needs.

For Sentinel-2 imagery products are available on the website https://scihub.copernicus.eu/dhus/ . Simple tools allow to target a particular region and sensing date. Therefore, a Sentinel-2 product covering the Siwa region can be downloaded quickly.

![Sentinel2_Scihub](./Images/15-Siwa_Oasis_S2B_MSIL1C_20210713_T35RLN.PNG)

Why downloading the product directly from the provider's website is not a solution of choice ?

* it will be downloaded directly on the personal computer of the user. The user will need to upload the product on his Jupyter Notebook environment

* Not convenient for making scripts that download many products automatically

* Depends on the user's internet speed for download and upload

* Does not allow to target a particular image inside the whole product

# Download a product using a DIAS

To avoid issues related to downloading and storing products, the European commission launched the Copernicus Data and Information Access Services (DIAS)

A DIAS is a cloud-based platforms. It provides centralised access to Copernicus data and information, as well as to processing tools.

5 DIAS exist, and all of them provide access to the data. 

![DIAS_offer.jpg](./Images/9-DIAS_Offer.jpg)

*CREODIAS* is the DIAS selected for the EO Africa R&D project. The purpose of this section is to download the product `S2B_MSIL1C_20210713T084559_N0301_R107_T35RLN_20210713T095529` using CREODIAS. 

## <a id='CREODIAS_search'></a> Search for datasets on CREODIAS

### EODAG embedded tool 

This JupyterHub environment comes with an embedded plugin that generates code using the *EODAG* library. 

EODAG (Earth Observation Data Access Gateway) is a Python package for searching and downloading remotely sensed images while offering a unified API for data access regardless of the data provider.

![EODAG_Overview.jpg](./Images/10-EODAG_overview.png)

In [ ]:
# Credentials path set in the below Bash environment
import os
os.environ["EODAG__CREODIAS_S3__DOWNLOAD__OUTPUTS_PREFIX"] = os.path.relpath('./Products/downloads')

The EODAG plugin allows to generate code related to the product.
The EODAG plugin can be used as shown in the screenshot below.

What are the characteristics of the product sought ? 

It is a Sentinel-2 tile that was sensed the 13/07/2021 , and covers the Siwa Oasis Area in Egypt.

The area is selected by clicking the black square at the top right of the interface.

![EODAG_Plugin.jpg](./Images/12-Plugin_EODAG_S2_Siwa_Oasis.png)

After clicking the search button, we obtain the following screen :

![EODAG_Plugin_Search](./Images/14-Reseach_result_S2_Siwa_Oasis.png)

Click the product banner then click apply on the top right of the interface that appears 

![EODAG_Plugin_search_Details](./Images/13-Reseach_result_S2_Siwa_Oasis_Details.png)

Automatically, a code snippet is generated in a new cell of the notebook. The code cell in question is displayed below, along with a few comments

In [ ]:
# The current code snippet was generated automatically by EODAG. Don't mind the red line that appears when executing the code cell, it is just a warning.

from eodag import EODataAccessGateway, setup_logging

setup_logging(1) # 0: nothing, 1: only progress bars, 2: INFO, 3: DEBUG

dag = EODataAccessGateway()

# The following line was added manually, it informs EODAG of the provider we want to use, CREODIAS in this case
dag.set_preferred_provider("creodias_s3")

geometry = "POLYGON ((25.224609 28.830238, 25.224609 29.430029, 25.933228 29.430029, 25.933228 28.830238, 25.224609 28.830238))"
search_results, total_count = dag.search(
  productType="S2_MSI_L1C",
  geom=geometry,
  start="2021-07-12",
  end="2021-07-14",
  cloudCover=100,
)

In [ ]:
# The rasterIO python library is used here to display the product (matplotlib is also popular)
import rasterio
import pprint

# The  line below ourput indicates that two producst were found in the search, and that they have indeed been downloaded using the Creodias provider
pprint.pprint(search_results)
# The targeted product is the first product, the one that covers the Siwa oasis area. The product is stored in the variable "product" that will be used in Section 2
product = search_results[0]

## Exploring the products

### Full product download 

**If you want to display a particular band without downloading the whole product, please go to section B.**

####  Download the whole products

The product of interest is now identified by EODAG. All that needs to be done now is to download the product. The EODAG function also unzips the file after it is downloaded.

In [ ]:
# The following line downloads the product
product_path = product.download()

####  Open and display image within a a fully downloaded product

In [ ]:
import glob
# We import the rasterIO library, that can read and display a Georeferenced image
from rasterio.plot import show

# Stores the product name, as it will be used in the next lines to construct the path to the image to show
product_name = product.properties['title']

# Looks for the Preview Image, stores the path. The preview image is quicker to load and display
path = glob.glob('./Products/downloads/'+ product_name + '/*ql.jpg')[0]

# Opens the image with rasterio, a python library
img = rasterio.open(path)

# Plots the preview image
show(img)

###  Display a particular image without downloading the whole product

EODag allows to "stream" a particular image contained in a targeted product, without actually downloading the whole file. 
Our product has already been identified by the search we conducted earlier : we know that the targeted product name is *S2B_MSIL1C_20210713T084559_N0301_R107_T35RLN_20210713T095529*

In [ ]:
# This first cell isolates the product's name, needed in the following cell.
import re
product_name = product.properties['title'].replace('.SAFE','')

search_results, _ = dag.search(id=product_name)

Let's try and stream band 10 of this Sentinel-2 product. Band 10 (1375 nm) corresponds to a very strong absorption, the lake area should then have a small luminance. 

In [ ]:
import os
from rasterio.crs import CRS

get_data_args = dict(
    crs=CRS.from_epsg(4326), 
    resolution=0.0006, 
    extent=search_results[0].geometry.bounds
)
# B10 stands for band 10, the 1380 nm SWIR band. The wavelength associated to band 10 hardly reflects water
band_B10 = [b for b in search_results[0].assets if "B10" in b and b.split(".")[-1] == "jp2"][0]
data = search_results[0].get_data(**{"band": band_B10, **get_data_args})

data

In [ ]:
#The following line plots Band 10 of the Sentinel-2 product
from matplotlib import pyplot
data.plot()

As expected, water surfaces are poorly reflected : pixel values on the lake are close to 0